# IBM Data Science Capstone Project: Settling in Sydney, Australia

## 1. Introduction

In this final project I choose to analyse the city of Sydney, AUS. Sydney is the largest city in Asutralia with more than 5 millions inhabitants. The city is very large with different neigbourhoods. All neigbourhoods offers different atmosphere. You have to ones close to the beach, the others closer to the city center or the business district. 
This analysis should help you find the best place for you to settle in Sydney.
It will show you:
 - What are the best locations as per infrstructure
 - What type of venues are there around - school, restaurants, parks, gyms, coffee-shops

According to your personal preference, you will be able to choose the best suited location/neigbourhood for you.

## 2. Data

As we decided to focus our analysis on Sydney, we need to get the data for Sydney and its suburbs. They can be found here [link](https://www.geonames.org/postal-codes/AU/NSW/new-south-wales.html). In  order to get the data we need to scrape the webpage. Where we need to be careful is to get the 2nd table of the webpage, as the first one refers to a search table. Once we get the relevant table, we need to clean it. Meaning droping the non-relevant columns, the columns with no values, renaming the columns and reseting the index. 

In a second part we will have to combine the latitude and longitude data to the table. As you will be able to see, the table from the webpage is already containing the latitude and longitude data. In an index section I will add the code I used to transform the table and only extract the latitude and longitude from it. The final geographical data are stored in a .csv file and will be imported and combined to the main table. 

The final table will then contain every neighbourhoods in Sydney, display its respective borough, zip code as well as its latittude and longitude.

The below steps get use to the final outcome, which will contain all the data relevant to pursuie our analysis.

### 2.1 Lets load required libraries

In [1]:
#importing librairies
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

### 2.2 Creation of the dataframe by scraping the relevant webpage

In [2]:
#importing more librairies
# import the library we use to open URLs

import urllib.request

In [3]:
# importing the table 

url = 'https://www.geonames.org/postal-codes/AU/NSW/new-south-wales.html'
page = urllib.request.urlopen(url)

In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 6.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup

In [6]:
pip install lxml

     |████████████████████████████████| 5.5MB 7.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
#getting the table from the below URL
url = 'https://www.geonames.org/postal-codes/AU/NSW/new-south-wales.html'
tables = pd.read_html(url)

In [8]:
#we choose the 2nd table from the webpage
sydney = tables[2]

In [9]:
#we want the name of the columns
sydney.columns

Index(['Unnamed: 0', 'Place', 'Code', 'Country', 'Admin1', 'Admin2', 'Admin3'], dtype='object')

In [11]:
#display table sydney
sydney

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Haymarket,2000,Australia,New South Wales,SYDNEY STREETS,NaN
1,NaN,-33.88/151.205,-33.88/151.205,-33.88/151.205,-33.88/151.205,-33.88/151.205,-33.88/151.205
2,2.0,Ultimo,2007,Australia,New South Wales,SYDNEY STREETS,NaN
3,NaN,-33.881/151.198,-33.881/151.198,-33.881/151.198,-33.881/151.198,-33.881/151.198,-33.881/151.198
4,3.0,Chippendale,2008,Australia,New South Wales,SYDNEY STREETS,NaN
...,...,...,...,...,...,...,...
396,199.0,St Ives Chase,2075,Australia,New South Wales,NaN,NaN
397,NaN,-33.709/151.162,-33.709/151.162,-33.709/151.162,-33.709/151.162,-33.709/151.162,-33.709/151.162
398,200.0,Normanhurst,2076,Australia,New South Wales,GOSFORD,NaN
399,NaN,-33.723/151.097,-33.723/151.097,-33.723/151.097,-33.723/151.097,-33.723/151.097,-33.723/151.097


### 2.3 Cleaning of the dataframe

In [10]:
#droping columns which we do not need
sydney.drop(columns=['Unnamed: 0', 'Country', 'Admin1', 'Admin3'], inplace=True)
#renaming column Admin 2 to Suburb
sydney.rename(columns={"Admin2": "Borough", "Place": "Neighbourhood", "Code": "Postal Code"}, inplace=True)
#dispplay table sydney
sydney

,Neighbourhood,Postal Code,Borough
0,Haymarket,2000,SYDNEY STREETS
1,-33.88/151.205,-33.88/151.205,-33.88/151.205
2,Ultimo,2007,SYDNEY STREETS
3,-33.881/151.198,-33.881/151.198,-33.881/151.198
4,Chippendale,2008,SYDNEY STREETS
...,...,...,...
396,St Ives Chase,2075,NaN
397,-33.709/151.162,-33.709/151.162,-33.709/151.162
398,Normanhurst,2076,GOSFORD
399,-33.723/151.097,-33.723/151.097,-33.723/151.097


In [11]:
#excludes every 2nd row starting from 1
sydney2 = sydney[sydney.index % 2 != 1].reset_index()  

In [12]:
#display new df sydney2
sydney2

,index,Neighbourhood,Postal Code,Borough
0,0,Haymarket,2000,SYDNEY STREETS
1,2,Ultimo,2007,SYDNEY STREETS
2,4,Chippendale,2008,SYDNEY STREETS
3,6,Pyrmont,2009,SYDNEY STREETS
4,8,Surry Hills,2010,SYDNEY STREETS
...,...,...,...,...
196,392,South Turramurra,2074,GOSFORD
197,394,Warrawee,2074,GOSFORD
198,396,St Ives Chase,2075,NaN
199,398,Normanhurst,2076,GOSFORD


In [13]:
sydney2.columns

Index(['index', 'Neighbourhood', 'Postal Code', 'Borough'], dtype='object')

In [14]:
#droping extra index column
sydney2.drop(columns=['index'], inplace=True)

In [15]:
#display new df sydney2
sydney2

,Neighbourhood,Postal Code,Borough
0,Haymarket,2000,SYDNEY STREETS
1,Ultimo,2007,SYDNEY STREETS
2,Chippendale,2008,SYDNEY STREETS
3,Pyrmont,2009,SYDNEY STREETS
4,Surry Hills,2010,SYDNEY STREETS
...,...,...,...
196,South Turramurra,2074,GOSFORD
197,Warrawee,2074,GOSFORD
198,St Ives Chase,2075,NaN
199,Normanhurst,2076,GOSFORD


In [16]:
#drop rows where Suburb is NaN
sydney2.dropna(subset=['Borough'], inplace=True)
#display new df sydney2
sydney2

,Neighbourhood,Postal Code,Borough
0,Haymarket,2000,SYDNEY STREETS
1,Ultimo,2007,SYDNEY STREETS
2,Chippendale,2008,SYDNEY STREETS
3,Pyrmont,2009,SYDNEY STREETS
4,Surry Hills,2010,SYDNEY STREETS
...,...,...,...
194,Turramurra,2074,GOSFORD
195,North Turramurra,2074,GOSFORD
196,South Turramurra,2074,GOSFORD
197,Warrawee,2074,GOSFORD


In [17]:
#reset index
sydney3= sydney2.reset_index()

In [18]:
#droping extra index column
sydney3.drop(columns=['index'], inplace=True)

In [19]:
#display new df sydney3
sydney3

,Neighbourhood,Postal Code,Borough
0,Haymarket,2000,SYDNEY STREETS
1,Ultimo,2007,SYDNEY STREETS
2,Chippendale,2008,SYDNEY STREETS
3,Pyrmont,2009,SYDNEY STREETS
4,Surry Hills,2010,SYDNEY STREETS
...,...,...,...
189,Turramurra,2074,GOSFORD
190,North Turramurra,2074,GOSFORD
191,South Turramurra,2074,GOSFORD
192,Warrawee,2074,GOSFORD


The dataframe sydney3 is the cleaned df we gonna used for the rest of the exercise.

### 2.4 Adding the latitude and longitude data 

In [20]:
#importing geocoder package. It is a module to convert an address into latitude and longitude values
import pip
!pip install geocoder
print("geocoder is now istalled!")

     |████████████████████████████████| 102kB 17.7MB/s ta 0:00:01
geocoder is now istalled!


In [21]:
#get the geocode data for sydney
syd_geocode = pd.read_csv ("sydney_geocode.csv")
print (syd_geocode)

     Postal Code  Latitude  Longitude
0           2000   -33.880    151.205
1           2007   -33.881    151.198
2           2008   -33.886    151.199
3           2009   -33.870    151.194
4           2010   -33.885    151.212
..           ...       ...        ...
195         2074   -33.704    151.149
196         2074   -33.753    151.111
197         2074   -33.729    151.123
198         2075   -33.709    151.162
199         2076   -33.723    151.097

[200 rows x 3 columns]


In [22]:
sydney3.dtypes

Neighbourhood    object
Postal Code      object
Borough          object
dtype: object

In [23]:
syd_geocode.dtypes

Postal Code      int64
Latitude       float64
Longitude      float64
dtype: object

In [24]:
sydney3['Postal Code'] = sydney3['Postal Code'].astype(float)

In [25]:
#join the two tables
syd_data = sydney3.join(syd_geocode.set_index('Postal Code'), on='Postal Code').reset_index()
syd_data

,index,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,0,Haymarket,2000.0,SYDNEY STREETS,-33.880,151.205
1,0,Haymarket,2000.0,SYDNEY STREETS,-33.861,151.204
2,0,Haymarket,2000.0,SYDNEY STREETS,-33.861,151.207
3,1,Ultimo,2007.0,SYDNEY STREETS,-33.881,151.198
4,2,Chippendale,2008.0,SYDNEY STREETS,-33.886,151.199
...,...,...,...,...,...,...
331,192,Warrawee,2074.0,GOSFORD,-33.732,151.130
332,192,Warrawee,2074.0,GOSFORD,-33.704,151.149
333,192,Warrawee,2074.0,GOSFORD,-33.753,151.111
334,192,Warrawee,2074.0,GOSFORD,-33.729,151.123


In [26]:
#droping extra index column
syd_data.drop(columns=['index'], inplace=True)
syd_data

,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,Haymarket,2000.0,SYDNEY STREETS,-33.880,151.205
1,Haymarket,2000.0,SYDNEY STREETS,-33.861,151.204
2,Haymarket,2000.0,SYDNEY STREETS,-33.861,151.207
3,Ultimo,2007.0,SYDNEY STREETS,-33.881,151.198
4,Chippendale,2008.0,SYDNEY STREETS,-33.886,151.199
...,...,...,...,...,...
331,Warrawee,2074.0,GOSFORD,-33.732,151.130
332,Warrawee,2074.0,GOSFORD,-33.704,151.149
333,Warrawee,2074.0,GOSFORD,-33.753,151.111
334,Warrawee,2074.0,GOSFORD,-33.729,151.123


In [27]:
syd_data['Postal Code'] = syd_data['Postal Code'].astype(object)
syd_data

,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,Haymarket,2000,SYDNEY STREETS,-33.880,151.205
1,Haymarket,2000,SYDNEY STREETS,-33.861,151.204
2,Haymarket,2000,SYDNEY STREETS,-33.861,151.207
3,Ultimo,2007,SYDNEY STREETS,-33.881,151.198
4,Chippendale,2008,SYDNEY STREETS,-33.886,151.199
...,...,...,...,...,...
331,Warrawee,2074,GOSFORD,-33.732,151.130
332,Warrawee,2074,GOSFORD,-33.704,151.149
333,Warrawee,2074,GOSFORD,-33.753,151.111
334,Warrawee,2074,GOSFORD,-33.729,151.123


In [28]:
syd_data.to_csv('syd_data.csv', index=False)  
print("Table saved as csv file!")

Table saved as csv file!


In [29]:
!pip install WeasyPrint #to able you to save as pdf

     |████████████████████████████████| 368kB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 34.6MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 32.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.7MB/s eta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/40/76/48/f1effadceea83b32e7d957dd0f92db4db8b537d7b72b4ef374
Successfully built cairocffi
ERROR: tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.1 which is incompatible.
ERROR: bleach 1.5.0 has requirement html5lib!=0.9999,!=0.99999,<0.99999999,>=0.999, but you'll have html5lib 1.1 which is incompatible.
  Found existing installation: html5lib 0.9999999
    Uninstalling html5lib-0.9999999:
      Successfully uninstalled html5lib-0.9999999


In [30]:
#save as pdf
from weasyprint import HTML
HTML(string=pd.read_csv('syd_data.csv').to_html()).write_pdf("syd_data.pdf")

We will use the Foursquare API in the next step, to get the venues for each neigbourhood. 

## 3. Import, display and analyse data thanks to the Foursquare API

In [31]:
#import missing laibrairies
import requests # library to handle requests
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### 3.1 Foursquare credentials

In [32]:
CLIENT_ID = 'ON22YSYNR3XM2QIUAGWCVDD1THX0P41WWHNW113JOTRCU1SB' # your Foursquare ID
CLIENT_SECRET = '0ELRHVJUBQDJA0FAK2WCVAC4AHVJTEJPCHJBC1BP3R4QGHBJ' # your Foursquare Secret
ACCESS_TOKEN = '3H0BUJ3ZHU5MU3NOAVPIK00HB4GGK2GKHTMC0DIDTEA0GBFG' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ON22YSYNR3XM2QIUAGWCVDD1THX0P41WWHNW113JOTRCU1SB
CLIENT_SECRET:0ELRHVJUBQDJA0FAK2WCVAC4AHVJTEJPCHJBC1BP3R4QGHBJ


### 3.2 Top 200 venues that are within a radius of 500 meters for each Neighbourhood

In [33]:
radius = 500
LIMIT = 225
venues = []

for lat, long, pin, post, city in zip(syd_data['Latitude'], syd_data['Longitude'], syd_data['Postal Code'], syd_data['Neighbourhood'], syd_data['Borough']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,            pin,            lat,             long,          city,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        venues_df = pd.DataFrame(venues)
        venues_df.head()



In [34]:
# define the column names
venues_df.columns = ['Neighbourhood', 'Postal Code', 'Latitude', 'Longitude', 'Borough', 'Venue_Name', 'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']

print(venues_df.shape)
venues_df.head()

(7460, 9)


,Neighbourhood,Postal Code,Latitude,Longitude,Borough,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
0,Haymarket,2000.0,-33.88,151.205,SYDNEY STREETS,Capitol Theatre,-33.879410,151.206248,Theater
1,Haymarket,2000.0,-33.88,151.205,SYDNEY STREETS,Do Dee Paidang,-33.880670,151.203018,Thai Restaurant
2,Haymarket,2000.0,-33.88,151.205,SYDNEY STREETS,Happy Chef 快樂廚,-33.878184,151.204372,Chinese Restaurant
3,Haymarket,2000.0,-33.88,151.205,SYDNEY STREETS,Chang Sabai Thai Massage & Spa,-33.879905,151.207024,Massage Studio
4,Haymarket,2000.0,-33.88,151.205,SYDNEY STREETS,Ho Jiak,-33.879303,151.203520,Malay Restaurant


In [35]:
venues_df.to_csv('venues_sydney_per_neighbourhood')

In [36]:
venues_df.groupby(['Neighbourhood','Postal Code','Borough']).count().head()

,,,Latitude,Longitude,Venue_Name,Venue_Latitude,Venue_Longitude,Venue_Category
Neighbourhood,Postal Code,Borough,,,,,,
Annandale,2038.0,LEICHHARDT,22,22,22,22,22,22
Armidale,2350.0,NSW NORTH COAST,19,19,19,19,19,19
Ashfield,2131.0,ST GEORGE,38,38,38,38,38,38
Auburn,2144.0,BANKSTOWN,19,19,19,19,19,19
Balmain,2041.0,LEICHHARDT,66,66,66,66,66,66


In [37]:
print('There are {} uniques categories.'.format(len(venues_df['Venue_Category'].unique())))

There are 308 uniques categories.


In [38]:
venues_df['Venue_Category'].unique()[:20]

array(['Theater', 'Thai Restaurant', 'Chinese Restaurant',
       'Massage Studio', 'Malay Restaurant', 'Japanese Restaurant',
       'Szechuan Restaurant', 'Hostel', 'Hotel', 'Sandwich Place',
       'Ice Cream Shop', 'Discount Store', 'Seafood Restaurant',
       'Taiwanese Restaurant', 'Tea Room', 'Food Court',
       'Hotpot Restaurant', 'Coffee Shop', 'Pastry Shop',
       'Korean BBQ Restaurant'], dtype=object)

### 3.3 Analysis of the venue categories for each Neighbourhood 

In [67]:
# one hot encoding
sydney_onehot = pd.get_dummies(venues_df[['Venue_Category']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
sydney_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 
sydney_onehot['Postal Code'] = venues_df['Postal Code'] 
sydney_onehot['Borough'] = venues_df['Borough'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(sydney_onehot.columns[-3:]) + list(sydney_onehot.columns[:-3])
sydney_onehot = sydney_onehot[fixed_columns]

print(sydney_onehot.shape)
sydney_onehot.head()

(7460, 311)


,Neighbourhood,Postal Code,Borough,ATM,Afghan Restaurant,African Restaurant,Airfield,American Restaurant,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,Haymarket,2000.0,SYDNEY STREETS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Haymarket,2000.0,SYDNEY STREETS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Haymarket,2000.0,SYDNEY STREETS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Haymarket,2000.0,SYDNEY STREETS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Haymarket,2000.0,SYDNEY STREETS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#group by Neighbourhood, Postal Code and Borough
sydney_grouped = sydney_onehot.groupby(["Neighbourhood", "Postal Code", "Borough"]).mean().reset_index()

print(sydney_grouped.shape)
sydney_grouped.head()

(191, 311)


,Neighbourhood,Postal Code,Borough,ATM,Afghan Restaurant,African Restaurant,Airfield,American Restaurant,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Volleyball Court,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,Annandale,2038.0,LEICHHARDT,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Armidale,2350.0,NSW NORTH COAST,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Ashfield,2131.0,ST GEORGE,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Auburn,2144.0,BANKSTOWN,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Balmain,2041.0,LEICHHARDT,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.015152,0.0,0.0,0.0,0.0,0.015152,0.0,0.0,0.0,0.0


### 3.4 Top 5 venues categories per postal code

In [41]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ["Neighbourhood", "Postal Code", "Borough"]
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
syd_ven_sorted = pd.DataFrame(columns=columns)
syd_ven_sorted['Neighbourhood'] = sydney_grouped['Neighbourhood']
syd_ven_sorted['Postal Code'] = sydney_grouped['Postal Code']
syd_ven_sorted['Borough'] = sydney_grouped['Borough']

for ind in np.arange(sydney_grouped.shape[0]):
    row_categories = sydney_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    syd_ven_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(syd_ven_sorted.shape)
syd_ven_sorted.head()

(191, 8)


,Neighbourhood,Postal Code,Borough,1st Venue,2nd Venue,3rd Venue,4th Venue,5th Venue
0,Annandale,2038.0,LEICHHARDT,Café,Grocery Store,Climbing Gym,Bakery,Bar
1,Armidale,2350.0,NSW NORTH COAST,Café,Chinese Restaurant,Pub,Department Store,Coffee Shop
2,Ashfield,2131.0,ST GEORGE,Dumpling Restaurant,Chinese Restaurant,Café,Malay Restaurant,Platform
3,Auburn,2144.0,BANKSTOWN,Café,Pizza Place,Supermarket,Coffee Shop,Turkish Restaurant
4,Balmain,2041.0,LEICHHARDT,Café,Pub,Park,Pizza Place,Sushi Restaurant


In [42]:
# Set manually to get proper fit in the map
address = 'Sydney'
latitude = -33.8688
longitude = 151.2093
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Sydney are -33.8688, 151.2093.


In [43]:
#adding latitude and longitude from syd_data to the sydney venue table.
sydney_all = syd_data.copy()
sydney_all = sydney_all.join(syd_ven_sorted[["Postal Code", "1st Venue"]].set_index("Postal Code"), on="Postal Code")
print(sydney_all.shape)
sydney_all.head()

(830, 6)


,Neighbourhood,Postal Code,Borough,Latitude,Longitude,1st Venue
0,Haymarket,2000,SYDNEY STREETS,-33.880,151.205,Café
0,Haymarket,2000,SYDNEY STREETS,-33.880,151.205,Café
0,Haymarket,2000,SYDNEY STREETS,-33.880,151.205,Café
1,Haymarket,2000,SYDNEY STREETS,-33.861,151.204,Café
1,Haymarket,2000,SYDNEY STREETS,-33.861,151.204,Café


### 3.5 Visualize the venues on a map

In [68]:
my_map = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, label1,common in zip(sydney_all['Latitude'], sydney_all['Longitude'], sydney_all['Neighbourhood'],sydney_all['1st Venue'] ):
    labelnew =  'Neighbourhood : {} , Top Existing Infrastructure  : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(my_map)  
my_map

## 4. Clustering the data

In [54]:
syd_ven_sorted['1st Venue'].unique()

array(['Café', 'Dumpling Restaurant', 'Bakery', 'Vietnamese Restaurant',
       'Bowling Alley', 'Sports Bar', 'Bookstore', 'Grocery Store',
       'Convenience Store', 'Fast Food Restaurant', 'Hotel', 'Brewery',
       'Supermarket', 'Malay Restaurant', 'Brazilian Restaurant',
       'Department Store', 'Chinese Restaurant', 'Pub', 'Bus Stop',
       'Harbor / Marina', 'Pizza Place', 'Park', 'Motel', 'Gym',
       'Tennis Court', 'Dessert Shop', 'Liquor Store', 'Golf Course',
       'Italian Restaurant', 'Indian Restaurant', 'Yoga Studio',
       'Lebanese Restaurant', 'Breakfast Spot', 'Lawyer', 'Sports Club',
       'Office', 'Diner', 'Coffee Shop', 'Tailor Shop', 'Clothing Store',
       'Electronics Store', 'Japanese Restaurant', 'Fish Market',
       'Gas Station'], dtype=object)

In [55]:
# one hot encoding
syd_ven_sorted_onehot = pd.get_dummies(syd_ven_sorted[['1st Venue']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
syd_ven_sorted_onehot['Neighbourhood'] = syd_ven_sorted['Neighbourhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(syd_ven_sorted_onehot.columns[-1:]) + list(syd_ven_sorted_onehot.columns[:-1])
syd_ven_sorted_onehot = syd_ven_sorted_onehot[fixed_columns]

print(syd_ven_sorted_onehot.shape)
syd_ven_sorted_onehot.head()

(191, 45)


,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bus Stop,Café,Chinese Restaurant,...,Park,Pizza Place,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio
0,Annandale,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Armidale,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Auburn,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Balmain,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
syd_ven_sorted_grouped = syd_ven_sorted_onehot.groupby(["Neighbourhood"]).sum().reset_index()

print(syd_ven_sorted_grouped.shape)
syd_ven_sorted_grouped.head()



(191, 45)


,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bus Stop,Café,Chinese Restaurant,...,Park,Pizza Place,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio
0,Annandale,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Armidale,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Auburn,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Balmain,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
syd_ven_sorted_grouped['Total Venues'] =  syd_ven_sorted_grouped[syd_ven_sorted_grouped.drop(['Neighbourhood'], axis=1).columns.values].sum(axis=1)

In [58]:
syd_ven_sorted_groupedmax = syd_ven_sorted_grouped[syd_ven_sorted_grouped['Total Venues'] == syd_ven_sorted_grouped['Total Venues'].max()]
print("Best place to stay within Sydney :")
syd_ven_sorted_groupedmax[['Neighbourhood', 'Total Venues']]
print(syd_ven_sorted_groupedmax.shape)

Best place to stay within Sydney :
(191, 46)


In [59]:
sydney_merged = syd_ven_sorted_grouped.copy()
sydney_merged = sydney_merged.join(syd_data[["Postal Code",'Latitude', 'Longitude', "Neighbourhood" ]].set_index("Neighbourhood"), on="Neighbourhood")

In [60]:
fixed_columns = list(sydney_merged.columns[-3:]) + list(sydney_merged.columns[:-3])
sydney_merged = sydney_merged[fixed_columns]

print(sydney_merged.shape)
sydney_merged.head()

(333, 49)


,Postal Code,Latitude,Longitude,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,...,Pizza Place,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio,Total Venues
0,2038,-33.882,151.171,Annandale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2350,-30.511,151.664,Armidale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2131,-33.888,151.126,Ashfield,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2144,-33.854,151.030,Auburn,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2041,-33.858,151.179,Balmain,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [61]:
# set number of clusters
kclusters = 3

sydney_grouped_clustering = sydney_merged[["Total Venues"]]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sydney_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0], dtype=int32)

In [62]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sydney_mergedfinal = sydney_merged.copy()
# add clustering labels
sydney_mergedfinal["Cluster Labels"] = kmeans.labels_
print(sydney_mergedfinal.shape)
sydney_mergedfinal.head() # check the last columns!

(333, 50)


,Postal Code,Latitude,Longitude,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,...,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio,Total Venues,Cluster Labels
0,2038,-33.882,151.171,Annandale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2350,-30.511,151.664,Armidale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2131,-33.888,151.126,Ashfield,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2144,-33.854,151.030,Auburn,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2041,-33.858,151.179,Balmain,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [63]:
# Set manually to get proper fit in the map
address = 'Sydney'
latitude = -33.8688
longitude = 151.2093
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Sydney are -33.8688, 151.2093.


In [64]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'orange',    'darkgreen',    'darkblue',    'black']
# add markers to map
markers_colors = []
for lat, lng, label1,common, cluster in zip(sydney_mergedfinal['Latitude'], sydney_mergedfinal['Longitude'], sydney_mergedfinal['Neighbourhood'],sydney_mergedfinal['Total infrastructure'] ,sydney_mergedfinal['Cluster Labels']):
    labelnew =  'Neighbourhood : {} , Total Venues : {}'.format(label1,common)
    label = folium.Popup( labelnew, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)
map_clusters

KeyError: 'Total infrastructure'

#### Cluster 1

In [65]:
sydney_mergedfinal.loc[sydney_mergedfinal['Cluster Labels'] == 0]

,Postal Code,Latitude,Longitude,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,...,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio,Total Venues,Cluster Labels
0,2038,-33.882,151.171,Annandale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2350,-30.511,151.664,Armidale,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2131,-33.888,151.126,Ashfield,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2144,-33.854,151.030,Auburn,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2041,-33.858,151.179,Balmain,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2256,-33.495,151.321,Woy Woy,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
188,2199,-33.905,151.021,Yagoona,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
189,2594,-34.314,148.300,Young,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
190,2017,-33.899,151.207,Zetland,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### Cluster 2

In [66]:
sydney_mergedfinal.loc[sydney_mergedfinal['Cluster Labels'] == 2]

,Postal Code,Latitude,Longitude,Neighbourhood,Bakery,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,...,Pub,Sports Bar,Sports Club,Supermarket,Tailor Shop,Tennis Court,Vietnamese Restaurant,Yoga Studio,Total Venues,Cluster Labels
